
# Programming for Geospatial Data Science

### Overview

- This coursework aims to create reproducible analyses to assess the following learning outcomes: programming core concepts, Python, packages for vector and raster data.

## Question 1: 2 points
### Answer the following questions:

1. What is a data structure?
2. Why is list comprehension fastest that a typical loop?
3. Let x = [1,'BBK', [3,'a'],-12.93]. What is the type of the variable x?
4. How many types of fuctions can we find in Python?
5. What is the difference between .loc and .iloc?

1. A data structure is a specialised way of organising, processing, retrieving and storing data.


2. List comprehension is faster than a typical loop because it is more concise, achieving the same result as for loops but in one line of code. Unlike for loops, list comprehension doesn't have to look up the list and then use the ```append()``` method for every iteration, making the process faster.


3. Nested list - a list that has a sublist as an element. 


4. There are 3 types of functions in Python. 
    1. **Built-in functions** - pre-defined functions.
    2. **Functions from packages** - pre-defined functions from packages.
    3. **User defined functions** - functions defined by the user to perform any specific task.


5. loc (location) and iloc (index location):
    - loc selects dataframe rows and columns with specific labels/names.
    - iloc selects dataframe rows and columns at specific integer index positions.

## Question 2: 3 points
### Load the file worldCities.csv into a dataframe named cities and code the following tasks:
1. Show the last ten records of cities
2. Show all the cities in the African continent
3. Show only the name and population of each city in the UK
4. Show all cities in Japan with a population > 1000 and less than 10000
5. Show a seaborn histrogram for the population distribution for all the cities in Spain.

###### 2.1. Show the last ten records of cities

In [ ]:
# importing pandas, geopandas and matplotlib libraries
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

# reading in the 'worldCities' csv as a dataframe stored in the variable named 'cities_org'
cities_org = pd.read_csv('data/worldCities.csv')

# exploring and preparing the dataframe
cities_org.shape
# the dataframe is made up of 39000 rows and 6 columns

# summarising for each column the number of null values in the dataframe
cities_org.isnull().sum()

In [ ]:
# displaying the null values in more detail
cities_org[cities_org['city'].isnull()]
cities_org[cities_org['population'].isnull()]

# removing the null values from the dataframe using the dropna() method storing the cleaned dataframe in the variable 'cities'
cities = cities_org.dropna()

cities.shape
# the 'cities' dataframe is made up of 38996 rows and 6 columns

In [ ]:
# verifying null values have been removed using an if/else statement 
if cities.isnull().values.any() == False:
    print('There are no null values within this dataframe')
else: 
    print ('Null values are present within this dataframe, please review')

In [ ]:
# using the tail method to return the last 10 rows of the dataframe
cities.tail(10)

###### 2.2. Show all the cities in the African continent

In [ ]:
# reading in the naturalearth_lowres data provided with geopandas 
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres')) 

# checking for null values in the 'world' dataframe:
world.isnull().values.any()

# creating an African countries dataframe where continent is equal to Africa
african_co = world[world.continent=='Africa']

# converting 'cities' dataframe to a geodataframe  
# using geopandas points_from_xy() to transform lat (Latitude) and lon (Longitude) into a geometry column of points 
# as per geopandas documentation: https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
cities_gdf = geopandas.GeoDataFrame(cities, geometry=geopandas.points_from_xy(cities.lon, cities.lat))

# setting and converting the CRS for both respective geodataframes 
cities_gdf = cities_gdf.set_crs("EPSG:4326")
african_co = african_co.to_crs("EPSG:4326")

# spatial join to identify African cities based on whether a 'cities_gdf' point geometry is within the an 'african_co'
# polygon geometry 
african_cities = geopandas.tools.sjoin(cities_gdf, african_co, predicate="within", how='inner')

# using the shape() function to identifies the number of rows within the 'african_cities' dataframe
african_cities.shape
# 2013 cities are located within countries that are assigned to the African continent 

# displaying African cities from the dataframe
african_cities['city']

# using list comprehension to create and display a list of African cities
african_cities_list = [city for city in african_cities['city']]
print(african_cities_list)

In [ ]:
# plotting African cities with geopandas to identify possible errors

# using the naturalearth_lowres data store within the 'world' variable as basemap for the plot
base = world.geometry.plot(figsize=(16, 12), color='lightgray', linewidth=0.5, edgecolor="black", alpha=0.4)
# plotting all cities in blue                           
cities_gdf.plot(ax=base, linewidth=1, color="blue", markersize=0.1)
# plotting African cities in red
african_cities.plot(ax=base, linewidth=1, color="red", markersize=0.1)
plt.show()

- Please note, the 'world' dataframe contains 177 countries and doesn't include all African countries. For example, Mauritius is African island east of the mainland continent but is not within in the 'world' dataframe and therefore not included in the spatial join.


- An alternative method to locate African cities would be to create a list of African countries and populate an additional continent column in the 'cities' dataframe with Africa if its country matches the country in the African countries list. 
- However, this relies of the country's name exactly matching in both the dataframe and the list.

###### 2.3. Show only the name and population of each city in the UK

In [ ]:
# selecting cities where country is 'UK' and storing the output in the variable 'uk_cities'
uk_cities = cities[cities.country=='UK']

# displaying only 'city' and 'population' columns from the 'uk_cities' dataframe
uk_cities[['city', 'population']]

In [ ]:
# creating and displaying a dictionary for UK cities and their population 
# code adapted from: https://cmdlinetips.com/2021/04/convert-two-column-values-from-pandas-dataframe-to-a-dictionary/
uk_cities_dic = dict(zip(uk_cities.city, uk_cities.population))
uk_cities_dic

###### 2.4. Show all cities in Japan with a population > 1000 and less than 10000

In [ ]:
# selecting cities where country is equal to 'Japan' and storing in the vairable 'japan_cities'
japan_cities = cities[cities.country=='Japan']

# using the query() function to display cities with a population greater than 1000 and less than 10000
japan_cities.query("1000 < population < 10000")

In [ ]:
# translating the queried Japanses cities into English and displaying output using list comprehension 
# code adapted from: https://medium.com/analytics-vidhya/how-to-translate-text-with-python-9d203139dcf5

# uncomment the following line to install 'deep-translator' if needed
#!pip install deep-translator

# import GoogleTanslator library 
from deep_translator import GoogleTranslator

# storing the queried dataframe into a vairbale named 'japan_cities_query'
japan_cities_query = japan_cities.query("1000 < population < 10000")

# using list comprehension to create a list of Japanses cities, tranlated into English, with a population 
# greater than 1000 and less than 10000
translated = [(GoogleTranslator(source='auto', target='en').translate(city)) for city in japan_cities_query['city']]

# printing the translated list 
print(translated)

###### 2.5. Show a seaborn histrogram for the population distribution for all the cities in Spain

In [ ]:
# importing seaborn library
# uncomment the following line to install 'seaborn' if needed
#!pip install seaborn
import seaborn as sns

# creating a new dataframe named 'spain_cities' where the country is equal to 'España'
spain_cities = cities[cities.country=='España']

# setting the plot style
sns.set_style('darkgrid')

# creating the frame and canvas for the plot
fig, ax = plt.subplots(1,figsize=(10,4))

# seaborn histogram plot using 'population' data from the 'spain_cities' dataframe 
# colour set to 'indigo', alpha (transparency) set to 0.6 and adding a kernel density estimation
sns.histplot(data=spain_cities, x="population", binwidth=10000, color='indigo', alpha=0.6, kde=True)

# adding a title and label to the x-axis
ax.set(title='Population of Spanish Cities', xlabel='Population');

In [ ]:
# using the describe function to show statistics of the population of Spanish cities
spain_cities["population"].describe()

# the distribution displays a postive skew, with the 75% of cities having a population of less than 25640 people

Real-world data is often skewed. For heavily skewed distributions, it’s better to define the bins in log space as per the [seaborn documentation](https://seaborn.pydata.org/generated/seaborn.histplot.html)

In [ ]:
# seaborn histogram plot using a log scale
fig, ax = plt.subplots(1,figsize=(10,4))
sns.histplot(data=spain_cities, x="population",log_scale=True, color='indigo', alpha=0.6)

# adding a title and label to the x-axis
ax.set(title='Population of Spanish Cities using a log scale', xlabel='Population');

In [ ]:
# subplot to compare the standard population distribution and population distribution using a log scale

# creating a histogram for each subplot
fig, ax = plt.subplots(1,2,figsize=(10,4))
sns.histplot(data=spain_cities, x='population', color='indigo', alpha=0.6, ax=ax[0])
sns.histplot(data=spain_cities, x='population', log_scale=True, color='indigo', alpha=0.6, ax=ax[1])

# adding titles and labels to the subplots
ax[0].set(title='Population of Spanish Cities', xlabel='Population');
ax[1].set(title='Population of Spanish Cities using a log scale', xlabel='Population');

## Question 3: 1 point
### Define and show the result of a function that print the result of $\sqrt n!$ , where $!$ represent the factorial of n. You need to show the result for all **ODD** numbers in the interval [1,20]

In [ ]:
#3.
# factorial of n
# ------------------
# the factorial of a number is the product of all the integers from 1 to that number
# n! = n * (n-1) * (n-2) * (n-3) * .... 3*2*1

# creating a user defined factorial function

# code for factorial function adapted from: https://www.programiz.com/python-programming/examples/factorial
# details on recursive functions: https://www.programiz.com/python-programming/recursion
def factorial(n):
    """This is a recursive function
    to find the factorial of an integer"""
    if n == 0:
        return 1
    # error handling 
    # if parameter is not an integer, raise an error
    elif isinstance(n, int) == False:
        raise RuntimeError('Parameter needs to be an integer, please try again')
    # if parameter is less than 0, raise an error
    elif n < 0:
        raise RuntimeError('Factorial of a negative number is undefined, please try again')
    # recursive call to the factorial() function
    else:
        return (n * factorial(n-1))

In [ ]:
# importing the math library
import math

# creating a user defined function that calculates the square route of the output of the factorial function 
def sqrt_factorial(n):
    """This is a function to calculate 
    the square root of a factorial integer"""
    fact = factorial(n)
    sqrt_fact = math.sqrt(fact)
    return sqrt_fact

In [ ]:
# creating a user defined function that calculates the square route of factorial numbers between a range of integers
# utilsing the factorial() and sqrt_factorial() functions

def sqrt_factorial_odd(range_lower, range_upper):
    """This is a function to calculate 
    the square root of a factorial integer,
    given a range values, only displaying the
    results of odd intervals between the
    input parameters""" 
    for n in range(int(range_lower), int(range_upper + 1)): # the range() function produces a sequence of numbers 
                                                            # up to the stop number but never includes the stop number.
                                                            # therefore, by adding 1 to upper range value it includes it as a 
                                                            # stop number rather than up to it
        # error handling 
        # raising errors if the range parameters aren't integers
        if isinstance(range_lower, int) == False:
            raise RuntimeError('Range parameter needs to be an integer, please try again')
        elif isinstance(range_upper, int) == False:
            raise RuntimeError('Range parameter needs to be an integer, please try again')
        # if range1 and range2 are both integer, then proceed
        else:
        # the modulus of an odd number is not equal to 0
        # determining whether n is odd by it's modulus not being equal to 0
            if n % 2 != 0:
                print(sqrt_factorial(n))
        

In [ ]:
# using the sqrt_factorial_odd() function to calculate the square route of odd integers factorials of  in the range of [1,20]
sqrt_factorial_odd(1, 20)

## Question 4: 2 points
### Create and test a user-defined function that that takes takes three input parameters: (a) a path to a raster file, (b) a path to a raster output file that will be created and (c) a numeric threshold.
### The function should reclassify the raster as a binary raster with True (1) if the value is >= threshold, and False (0) <br> if the value is < threshold and save this reclassification on a new raster in the specified path provided at (b). The function returns True when completing correctly.

#### You can test your function with any of the raster provided in the data folder.

In [ ]:
# importing relevant libraries

# uncomment the following line to install 'rasterio' if needed
#!pip install rasterio

import rasterio 
import rasterio.plot
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import TwoSlopeNorm

In [ ]:
# creating a user-defined raster threshold function
# code adapted from lecture 6 content 

def raster_threshold(in_path, out_path, threshold):
    """This function reclassifies a raster as a
    binary raster with True (1) or False (0) based
    on a threshold value and saves the reclassified 
    raster in a specified path"""

    # error handing
    # raising error if paths aren't strings and the threshold isn't an integer or float
    for path in [in_path, out_path]:
        if isinstance(path, str) == False:
            raise RuntimeError('Path needs to be a valid directory, please try again')
    if isinstance(threshold, (float, int)) == False:
        raise RuntimeError('Threshold value must be a number, please try again')
    
    else:
        # using the in_path parameter and rastio.open() function to access the raster as an object and store it in the
        # variable named 'in_data'
        # masked=True tells rasterio to ignore the NULL values
        in_data = rasterio.open(in_path, mask=True)

        # reading in the first band of the raster and storing in the variable named 'vals'
        vals = in_data.read(1, masked=True)

        # copying the data matrix to preserve the original data
        recl_vals = vals.copy()
        # reclassification base on threshold parameter value
        recl_vals[vals >= threshold] = 1
        recl_vals[vals < threshold] = 0

        # saving the reclassified raster file, using metadata from the input files
        out_data = rasterio.open(out_path, 'w', 
            driver='GTiff', # output format
            height=recl_vals.shape[0], # size of matrix
            width=recl_vals.shape[1], # size of matrix
            count=1, # number of bands
            dtype=recl_vals.dtype, # type of data (e.g., floating point)
            crs=in_data.crs, # CRS (e.g., Lambert, WGS84, UTM, etc.)
            nodata=in_data.nodata, # value used to represent NO DATA
            transform=in_data.transform # geocoordinate transformation
        )

        out_data.write(recl_vals, 1)
        out_data.close()
    
    # the function returns True when completed correctly
    return True

###### Testing the raster_threshold() function 

In [ ]:
# plot_raster() function - user defined function adapted from: https://geoscripting-wur.github.io/PythonRaster
# note, the default values (Blues, 10, 10).
def plot_raster(rast, val_matrix, plot_title, value_label, cmap='Blues', width=10, height=10, diverge_zero=False):
    """Plots a rasterio raster with sensible settings and with a legend .
        @ rast: rasterio file (used to read the geocoordinates)
        @ val_matrix: extracted values (used to read the raster values)
        @ plot_title: title of the whole figure
        @ value_label: quantity being displayed
        @ diverge_zero: true if using a divergent cmap to center colour map on zero
    """
    # defining the plot frame and canvas 
    fig, ax = plt.subplots(figsize=(6,6))
    # image_hidden is a hack to show the legend
    if diverge_zero:
        image_hidden = ax.imshow(val_matrix, cmap=cmap, norm=TwoSlopeNorm(0)) #imshow is a python function that allows you to plot images from a file, #twoslopenorm() function is used to normalise the values
    else:
        image_hidden = ax.imshow(val_matrix, cmap=cmap)

    ax.clear()
    # plot raster: rast.transform allows the system to show geocoordinates
    if diverge_zero:
        rast_plot = rasterio.plot.show(val_matrix, cmap=cmap, ax=ax, transform=rast.transform, norm=TwoSlopeNorm(0))
    else: 
        rast_plot = rasterio.plot.show(val_matrix, cmap=cmap, ax=ax, transform=rast.transform)
    # set plot title
    ax.set_title(plot_title, fontsize=14)
    # show legend with label
    # hack to fix height
    im_ratio = val_matrix.shape[0]/val_matrix.shape[1] 
    #plt.colorbar(im,fraction=0.046*im_ratio, pad=0.04)
    cbar = fig.colorbar(image_hidden, ax=ax, fraction=0.046*im_ratio, pad=0.04)
    cbar.ax.set_ylabel(value_label, rotation=270)
    cbar.ax.get_yaxis().labelpad = 15
    #ax.set_axis_off() # enable/disable axes
    plt.show()

In [ ]:
# testing the raster_threshold() function with 5 different threshold values using a for loop
# working with the Himalayan Mountain range raster
thresholds = [100,500,1000,2000,3000]

for n in thresholds:
    raster_threshold('data/dem_srtm_1km_himalayas_2009.tif', 'output/test'+ str(n) +'.tif', n)

In [ ]:
# plotting all the test rasters using a for loop and the plot_raster() function 

for n in thresholds:
    test_raster = rasterio.open('output/test'+str(n)+'.tif', mask=True)
    vals = test_raster.read(1, masked=True)
    # setting the plot style to a white using seaborn
    # 'seaborn comes with a number of customized themes and a high-level interface for controlling the look of matplotlib figures'
    # as per the seaborn documentation: https://seaborn.pydata.org/tutorial/aesthetics.html
    sns.set_style("white")
    # plotting the 
    plot_raster(test_raster, vals, 'Test '+str(n) + 'm', 'Reclassified Raster Cell Values (1=max) where threshold = '+ str(n) + 'm', cmap='Reds', width=5, height=5)


## Question 5: 4 points
### Using the same dataframe from question 2, do the following tasks:
1. Create a geodataframe building the geometry from the columns lat, lon  (Hint: https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html)
2. Color the points according to population distribution. You decide the number of classes, for example
* Blue for pop<10000;
* Red for 10000<pop<=100000;
* Green for 100000<pop<=1000000;
* Black for pop>1000000 <br>
You can use https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3 to decide the colours. 
3. Your map need to include a legend for the classes.
4. Save your map to a pdf file.
5. As a base world map, you can use this line <br> **world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))** <br> and follow as a guidance the same link at point 1

###### 5.1. Create a geodataframe building the geometry from the columns lat, lon

In [ ]:
# importing the relevant libraries 
import pandas as pd
import geopandas 
import matplotlib.pyplot as plt

# reading in the 'worldCities' csv as a dataframe stored in the vairable named 'cities_ord'
cities_org = pd.read_csv('data/worldCities.csv')

# using geopandas points_from_xy() to transform lat (Latitude) and lon (Longitude) into a into a geometry column of points 
# as per geopandas documentation https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
cities_org = geopandas.GeoDataFrame(cities, geometry=geopandas.points_from_xy(cities.lon, cities.lat))

# removing null values from the geodataframe using the dropna() method
cities_gdf = cities_org.dropna()

# displaying the cleaned geodataframe
cities_gdf

###### 5.2. Color the points according to population distribution

In [ ]:
# mapclassify library allows classification schemes for choropleth maps
# uncomment the following line to install 'mapclassify' if needed
#!pip install mapclassify
import mapclassify 

# plotting the data with quantile binning scheme and 6 classes
# quantile binning aims to assign the same number of observations to each bin
cities_gdf.plot(column='population', scheme='Quantiles', k=6)
plt.show()

###### 5.3. Your map need to include a legend for the classes

In [ ]:
# adding a legend, changing the size of the points and colour to 'plasma'
# displaying a legend with the threshold values as integers
cities_gdf.plot(column='population', figsize=(16, 12), s=1, scheme='Quantiles', cmap="plasma", k=6, legend=True,
         legend_kwds={'fmt': '{:.0f}','loc': 'lower left', 'title': 'Population'})
# adding a title
plt.title("Population distribution of world cities")
plt.show()

###### 5.4. Save your map to a pdf file

In [ ]:
# plotting the data from the geodataframe
cities_gdf.plot(column='population', figsize=(16, 12), s=1, scheme='Quantiles', cmap="plasma", k=6, legend=True,
         legend_kwds={'fmt': '{:.0f}','loc': 'lower left', 'title': 'Population'})
# adding a title
plt.title("Population distribution of world cities")

# saving map as a pdf in the output directory 
plt.savefig('output/city_popualtion.pdf')

###### 5.5. Add a base world map

In [ ]:
# reading in a basemap using the naturalearth_lowres data provided with geopandas
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# storing the basemap in the variable 'ax'
ax = world.plot(figsize = (16,12), color='lightgray', alpha = 0.5, edgecolor="gray", linewidth=1)

# plotting cities geodataframe with basemap
cities_gdf.plot(ax = ax, column='population', s=1, scheme='Quantiles', cmap="plasma", k=6, legend=True,
         legend_kwds={'fmt': '{:.0f}','loc': 'lower left', 'title': 'Population'})
# adding a title
plt.title("Population distribution of world cities")
plt.show()

###### Exploring the distribution of world cities with a interactive visualisation using pydeck

In [ ]:
# height of the column is determined by the population 

# code adapted from: 
# https://pydeck.gl/layer.html
# https://deckgl.readthedocs.io/en/latest/gallery/column_layer.html
# and https://discuss.streamlit.io/t/how-to-make-pydeck-chart-hexagon-layer-bar-height-based-on-column-value/3959

# uncomment the following line to install 'pydeck' if needed
#!pip install pydeck
import pydeck as pdk

# setting the viewport location and zoom
view_state = pdk.ViewState(
    longitude=0,
    latitude=45,
    zoom=3,
    min_zoom=3,
    max_zoom=10,
    pitch=75,
    bearing=0,)

# assigning the layer details to the variable 'column_layer'
column_layer = pdk.Layer(
    "ColumnLayer",
    data=cities_gdf,
    get_position=["lon", "lat"],
    get_elevation="population",
    elevation_scale=0.1,
    radius=1000,
    get_fill_color=['population'],
    pickable=True,
    auto_highlight=True,)

# preparing tooltip popup to display city and population details
tooltip = {
    "html": "<b>{city}</b> has a population of <b>{population}</b>",
    "style": {"background": "grey", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000"},}

# combining the layer vairables and rendering the viewport
r = pdk.Deck(column_layer, initial_view_state=view_state,tooltip=tooltip)
r

# hold CTR to pan the map

# Good luck!

### End of Notebook